In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc=SparkContext()
spark = SparkSession(sparkContext=sc)

# Column Expression

**select()** , **groupby()**, **orderby()** from pyspark.sql.DataFrame

## Load data

In [2]:
mtcars = spark.read.csv('../../data/mtcars.csv',inferSchema=True,header=True)
mtcars = mtcars.withColumnRenamed("_c0",'model')
mtcars.show(5)

+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|            model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|        Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
|    Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|       Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|   Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|
|Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
only showing top 5 rows



## Use dot(.) to select column from DataFrame

In [3]:
mpg_col = mtcars.mpg
mpg_col

Column<b'mpg'>

In [4]:
mtcars.select(mpg_col * 100).show(5)

+-----------+
|(mpg * 100)|
+-----------+
|     2100.0|
|     2100.0|
|     2280.0|
|     2140.0|
|     1870.0|
+-----------+
only showing top 5 rows



The pyspark.sql.Column has many methods that acts on a column and returns a column instance.

In [5]:
mtcars.select(mtcars.gear.isin([2,3])).show(5)

+----------------+
|(gear IN (2, 3))|
+----------------+
|           false|
|           false|
|           false|
|            true|
|            true|
+----------------+
only showing top 5 rows



# Boolean column Expression
## between(): true/false if the column value is between a given range

In [3]:
mpg_between = mtcars.cyl.between(4,6)
mpg_between

Column<b'((cyl >= 4) AND (cyl <= 6))'>

In [4]:
mtcars.select(mtcars.cyl,mpg_between).show(5)

+---+---------------------------+
|cyl|((cyl >= 4) AND (cyl <= 6))|
+---+---------------------------+
|  6|                       true|
|  6|                       true|
|  4|                       true|
|  6|                       true|
|  8|                      false|
+---+---------------------------+
only showing top 5 rows



## contains(): true/false if the column value contains a string

In [5]:
model_contains = mtcars.model.contains('Ho')
model_contains

Column<b'contains(model, Ho)'>

In [6]:
mtcars.select(mtcars.model,model_contains).show(5)

+-----------------+-------------------+
|            model|contains(model, Ho)|
+-----------------+-------------------+
|        Mazda RX4|              false|
|    Mazda RX4 Wag|              false|
|       Datsun 710|              false|
|   Hornet 4 Drive|               true|
|Hornet Sportabout|               true|
+-----------------+-------------------+
only showing top 5 rows



## endswith(): true/false if the column value ends with a string

In [7]:
model_endswith = mtcars.model.endswith('t')
model_endswith

Column<b'endswith(model, t)'>

In [8]:
mtcars.select(mtcars.model,model_endswith).show(5)

+-----------------+------------------+
|            model|endswith(model, t)|
+-----------------+------------------+
|        Mazda RX4|             false|
|    Mazda RX4 Wag|             false|
|       Datsun 710|             false|
|   Hornet 4 Drive|             false|
|Hornet Sportabout|              true|
+-----------------+------------------+
only showing top 5 rows



## isNotNull(): true/false if the column value is Not Null

In [9]:
from pyspark.sql import Row
df = spark.createDataFrame([Row(name='Tom',height=80),Row(name='Alice',height=None)])
df.show()

+------+-----+
|height| name|
+------+-----+
|    80|  Tom|
|  null|Alice|
+------+-----+



In [10]:
height_isnotnull = df.height.isNotNull()
height_isnotnull

Column<b'(height IS NOT NULL)'>

In [11]:
df.select(df.height,height_isnotnull).show()

+------+--------------------+
|height|(height IS NOT NULL)|
+------+--------------------+
|    80|                true|
|  null|               false|
+------+--------------------+



## isNull( ): true/false if the column value is Null

In [12]:
height_isnull = df.height.isNull()
height_isnull

Column<b'(height IS NULL)'>

In [14]:
df.select(df.height,height_isnull).show()

+------+----------------+
|height|(height IS NULL)|
+------+----------------+
|    80|           false|
|  null|            true|
+------+----------------+



## like(): true/false if the column value matches a pattern based on *SQL LIKE*.

In [15]:
model_like = mtcars.model.like('Ho%')
model_like

Column<b'model LIKE Ho%'>

In [16]:
mtcars.select(mtcars.model,model_like).show(5)

+-----------------+--------------+
|            model|model LIKE Ho%|
+-----------------+--------------+
|        Mazda RX4|         false|
|    Mazda RX4 Wag|         false|
|       Datsun 710|         false|
|   Hornet 4 Drive|          true|
|Hornet Sportabout|          true|
+-----------------+--------------+
only showing top 5 rows



## rlike( ): true/false if the column value matches a pattern based on a *SQL RLIKE* (like with regex)

In [17]:
model_rlike = mtcars.model.rlike("t$")
model_rlike

Column<b'model RLIKE t$'>

In [19]:
mtcars.select(mtcars.model,model_rlike).show()

+-------------------+--------------+
|              model|model RLIKE t$|
+-------------------+--------------+
|          Mazda RX4|         false|
|      Mazda RX4 Wag|         false|
|         Datsun 710|         false|
|     Hornet 4 Drive|         false|
|  Hornet Sportabout|          true|
|            Valiant|          true|
|         Duster 360|         false|
|          Merc 240D|         false|
|           Merc 230|         false|
|           Merc 280|         false|
|          Merc 280C|         false|
|         Merc 450SE|         false|
|         Merc 450SL|         false|
|        Merc 450SLC|         false|
| Cadillac Fleetwood|         false|
|Lincoln Continental|         false|
|  Chrysler Imperial|         false|
|           Fiat 128|         false|
|        Honda Civic|         false|
|     Toyota Corolla|         false|
+-------------------+--------------+
only showing top 20 rows



## startswith( ): true/false if the column value starts with a string

In [20]:
model_starswith = mtcars.model.startswith("Merc")
model_starswith

Column<b'startswith(model, Merc)'>

In [21]:
mtcars.select(mtcars.model,model_starswith).show()

+-------------------+-----------------------+
|              model|startswith(model, Merc)|
+-------------------+-----------------------+
|          Mazda RX4|                  false|
|      Mazda RX4 Wag|                  false|
|         Datsun 710|                  false|
|     Hornet 4 Drive|                  false|
|  Hornet Sportabout|                  false|
|            Valiant|                  false|
|         Duster 360|                  false|
|          Merc 240D|                   true|
|           Merc 230|                   true|
|           Merc 280|                   true|
|          Merc 280C|                   true|
|         Merc 450SE|                   true|
|         Merc 450SL|                   true|
|        Merc 450SLC|                   true|
| Cadillac Fleetwood|                  false|
|Lincoln Continental|                  false|
|  Chrysler Imperial|                  false|
|           Fiat 128|                  false|
|        Honda Civic|             

In [22]:
from pyspark.sql import functions as F

## abs( ): create column expression that returns absolute values of a column

In [23]:
from pyspark.sql import Row

In [24]:
df = sc.parallelize([Row(x=1),Row(x=-1),Row(x=-2)]).toDF()
df.show()

+---+
|  x|
+---+
|  1|
| -1|
| -2|
+---+



In [25]:
x_abs = F.abs(df.x)
x_abs

Column<b'abs(x)'>

In [26]:
df.select(df.x,x_abs).show()

+---+------+
|  x|abs(x)|
+---+------+
|  1|     1|
| -1|     1|
| -2|     2|
+---+------+



## corr( ): create column expression that returns pearson corrleation coefficient between two columns

In [27]:
drat_wt_corr = F.corr(mtcars.drat,mtcars.wt)
drat_wt_corr

Column<b'corr(drat, wt)'>

In [28]:
mtcars.select(drat_wt_corr).show()

+-------------------+
|     corr(drat, wt)|
+-------------------+
|-0.7124406466973717|
+-------------------+



## array( ): create column expression that merge multiple column values into an array

In [29]:
cols = [eval('mtcars.'+col) for col in mtcars.columns[1:]]
cols

[Column<b'mpg'>,
 Column<b'cyl'>,
 Column<b'disp'>,
 Column<b'hp'>,
 Column<b'drat'>,
 Column<b'wt'>,
 Column<b'qsec'>,
 Column<b'vs'>,
 Column<b'am'>,
 Column<b'gear'>,
 Column<b'carb'>]

In [30]:
cols_array = F.array(cols)
cols_array

Column<b'array(mpg, cyl, disp, hp, drat, wt, qsec, vs, am, gear, carb)'>

In [31]:
mtcars.select(cols_array).show(truncate=False)

+-----------------------------------------------------------------+
|array(mpg, cyl, disp, hp, drat, wt, qsec, vs, am, gear, carb)    |
+-----------------------------------------------------------------+
|[21.0, 6.0, 160.0, 110.0, 3.9, 2.62, 16.46, 0.0, 1.0, 4.0, 4.0]  |
|[21.0, 6.0, 160.0, 110.0, 3.9, 2.875, 17.02, 0.0, 1.0, 4.0, 4.0] |
|[22.8, 4.0, 108.0, 93.0, 3.85, 2.32, 18.61, 1.0, 1.0, 4.0, 1.0]  |
|[21.4, 6.0, 258.0, 110.0, 3.08, 3.215, 19.44, 1.0, 0.0, 3.0, 1.0]|
|[18.7, 8.0, 360.0, 175.0, 3.15, 3.44, 17.02, 0.0, 0.0, 3.0, 2.0] |
|[18.1, 6.0, 225.0, 105.0, 2.76, 3.46, 20.22, 1.0, 0.0, 3.0, 1.0] |
|[14.3, 8.0, 360.0, 245.0, 3.21, 3.57, 15.84, 0.0, 0.0, 3.0, 4.0] |
|[24.4, 4.0, 146.7, 62.0, 3.69, 3.19, 20.0, 1.0, 0.0, 4.0, 2.0]   |
|[22.8, 4.0, 140.8, 95.0, 3.92, 3.15, 22.9, 1.0, 0.0, 4.0, 2.0]   |
|[19.2, 6.0, 167.6, 123.0, 3.92, 3.44, 18.3, 1.0, 0.0, 4.0, 4.0]  |
|[17.8, 6.0, 167.6, 123.0, 3.92, 3.44, 18.9, 1.0, 0.0, 4.0, 4.0]  |
|[16.4, 8.0, 275.8, 180.0, 3.07, 4.07, 17.4, 0.0